In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from math import sqrt
import time

In [2]:

#current_time=df.loc[0,'time']

# base_power = 0.99 
# fluctuation_threshold = 0.01
# block_time 
# current_time = df.loc[0,'time']    # Time is in minutes

s = pd.Series([24, 22, 21, 23, 21, 35, 36, 33, 34, 33, 34, 35, 34, 20, 19, 19, 18, 20, 40, 41])
time1 = pd.date_range('1/1/2000', periods=20, freq='1s')
pf = pd.Series([0.8, 0.8, 0.8, 0.8, 0.8, 0.9, 0.9, 0.9, 0.9, 0.95, 0.95, 0.95, 0.8, 0.8, 0.9, 0.95, 0.8, 0.9, 0.95, 0.95])
vol = pd.Series([415, 416, 415, 415, 415, 416, 415, 415, 416, 415, 415, 415, 416, 416, 415, 416, 415, 416, 416, 416])
thd = pd.Series([85, 45, 38, 38, 38, 45, 38, 38, 38, 45, 52, 38, 38, 45, 52, 38, 38, 45, 52, 38])

df= pd.DataFrame({'time':time1,'number':s,'pf':pf, 'vol':vol, 'thd':thd})
df.thd=df.thd/100
print(df)

                  time  number    pf  vol   thd
0  2000-01-01 00:00:00      24  0.80  415  0.85
1  2000-01-01 00:00:01      22  0.80  416  0.45
2  2000-01-01 00:00:02      21  0.80  415  0.38
3  2000-01-01 00:00:03      23  0.80  415  0.38
4  2000-01-01 00:00:04      21  0.80  415  0.38
5  2000-01-01 00:00:05      35  0.90  416  0.45
6  2000-01-01 00:00:06      36  0.90  415  0.38
7  2000-01-01 00:00:07      33  0.90  415  0.38
8  2000-01-01 00:00:08      34  0.90  416  0.38
9  2000-01-01 00:00:09      33  0.95  415  0.45
10 2000-01-01 00:00:10      34  0.95  415  0.52
11 2000-01-01 00:00:11      35  0.95  415  0.38
12 2000-01-01 00:00:12      34  0.80  416  0.38
13 2000-01-01 00:00:13      20  0.80  416  0.45
14 2000-01-01 00:00:14      19  0.90  415  0.52
15 2000-01-01 00:00:15      19  0.95  416  0.38
16 2000-01-01 00:00:16      18  0.80  415  0.38
17 2000-01-01 00:00:17      20  0.90  416  0.45
18 2000-01-01 00:00:18      40  0.95  416  0.52
19 2000-01-01 00:00:19      41  0.95  41

In [3]:

class PowerFactorLoss_KPI ():
    block_values = []
    
    def __init__(self, base_power, fluctuation_threshold):
        self.base_power=base_power
        self.fluctuation_threshold=fluctuation_threshold
    
    
    
    def initialize (self, power_t, power_factor_t, current_time, thd_t, mean_volt_t):
        self.block_counter=1
        self.block_average_power=power_t
        self.block_average_pf=power_factor_t
        self.block_start_time=current_time 
        self.block_average_thd=thd_t
        self.block_average_voltage=mean_volt_t
        return

    
    
    def __power_fluctuation(self, power_t):
        if (abs(power_t-self.block_average_power)/self.block_average_power <= self.fluctuation_threshold):
            return (0)
        else: 
            return (1)

    
    def __update_average_power (self, power_t):
        return ((self.block_average_power*self.block_counter + power_t)/(self.block_counter+1))
    
    
    def __update_average_power_factor (self, power_factor_t):
        return ((self.block_average_pf*self.block_counter + power_factor_t)/(self.block_counter+1))
    
    
    def __update_average_thd (self, thd_t):
        return ((self.block_average_thd*self.block_counter + thd_t)/(self.block_counter+1))
    
    
    def __update_average_voltage (self, mean_volt_t):
        return ((self.block_average_voltage*self.block_counter + mean_volt_t)/(self.block_counter+1))
    
    #def __block_average_loss (self):
        #return ((self.block_average_power*1000*(1-(1/sqrt(1-(self.block_average_thd*self.block_average_thd)))))/(sqrt(3)*self.block_average_voltage))
    
    
#     def calculate_power_loss(self):
#         n = len(self.block_values)
#         kVAh_loss = np.sum(((1/a[2])-(1/self.base_power))*((a[1]*a[0])/3600) for a in self.block_values)
#         kWAh_loss = kVAh_loss * np.sum([a[2]*(a[0]/60.0) for a in self.block_values])/(np.sum([a[0] for a in self.block_values])/60.0)
#         kVArh_lag = np.sqrt(kVAh_loss**2 - kWAh_loss**2)
#         kVArh_lead = np.sqrt(kVAh_loss**2 - (kVAh_loss*self.base_power)**2)
#         cycle_power_factor = kWAh_loss / sqrt(kVAh_loss**2 + (kVArh_lag + kVArh_lead)**2)
          
#         return ({'kVAh_loss':kVAh_loss, 'kWAh_loss':kWAh_loss, 'cycle_power_factor':cycle_power_factor})
        
        
                 
    def check (self, power_t, power_factor_t, current_time, thd_t, mean_volt_t):
        if self.__power_fluctuation(power_t):
            self.block_values.append([(current_time - self.block_start_time).total_seconds(), self.block_average_power, self.block_average_pf, self.block_average_thd, self.block_average_voltage,])
            self.initialize (power_t, power_factor_t, current_time, thd_t, mean_volt_t)
        else:
            self.block_average_power = self.__update_average_power (power_t)
            self.block_average_pf = self.__update_average_power_factor (power_factor_t)
            self.block_average_thd = self.__update_average_thd (thd_t)
            self.block_average_voltage = self.__update_average_voltage (mean_volt_t)
            self.block_counter += 1

In [4]:
obj = PowerFactorLoss_KPI(0.99, 0.1)


obj.initialize(df.loc[0,'number'], df.loc[0,'pf'], df.loc[0,'time'], df.loc[0,'thd'],  df.loc[0,'vol'])


for i in range(1,len(df)-1):
    obj.check (df.loc[i,'number'],df.loc[i,'pf'],df.loc[i,'time'],df.loc[i,'thd'],df.loc[i,'vol'])
     

df_x=pd.DataFrame(obj.block_values)
df_x

def function_cal(row):
   
    temp_=  1 - ( 1/sqrt( 1 - (row[3]**2)))
    return (( (row[1]*1000) *( temp_) /  ((sqrt(3)*row[4]))))   


df_x['5']=df_x.apply(lambda x:function_cal(x),axis=1)
df_x.columns=['Time_interval', 'Avg_power', 'Avg_pf', 'Avg_thd', 'Avg_vol', 'Loss']


df_x

,Time_interval,Avg_power,Avg_pf,Avg_thd,Avg_vol,Loss
0,5.0,22.20,0.80000,0.488,415.200,-4.497143
1,8.0,34.25,0.90625,0.415,415.375,-4.718532
2,5.0,19.20,0.87000,0.436,415.600,-2.965377


In [5]:
kvah_loss_pf = 0
kvah_loss_thd = 0
kvah_loss_total = 0
kVArh_lag = 0
kVArh_lead = 0 
cycle_power_factor = 0

for i in range(0, len(df_x)):
     kvah_loss_pf = kvah_loss_pf + ((df_x.loc[i,'Avg_power']*df_x.loc[i,'Time_interval'])/3600)*((1/df_x.loc[i,'Avg_pf'])-(1/0.99))
     kvah_loss_thd = kvah_loss_thd +(sqrt(3)*(df_x.loc[i,'Avg_vol']*df_x.loc[i,'Loss']*df_x.loc[i,'Time_interval'])/3600000)
     kvah_loss_total = kvah_loss_pf + kvah_loss_thd

print(kvah_loss_pf, kvah_loss_thd, kvah_loss_total) 

0.018216963698285526 -0.015000426975487525 0.003216536722798001


In [6]:
# Initialize the KPI

# BASE_POWER, POWER_FLUCTUATION_ACCEPTED, TIME_BLOCK = 0.99, 0.05, 30     # Time is in minutes
# PowerFactorLoss_KPI (BLOCK_POWER, POWER_FLUCTUATION_ACCEPTED, TIME_BLOCK, current_time)

# PowerFactorLoss_KPI.initialize (instant_power, instant_power_factor, current_time)

# ...loop of the streaming data...

# PowerFactorLoss_KPI.check(instant_power, instant_power_factor, current_time)

# ......

